# Imports

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, MaxPooling2D, GlobalAveragePooling2D, Dense, MaxPool2D
from tensorflow.keras.utils import to_categorical
from keras.layers import ReLU, GlobalAvgPool2D, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from PIL import Image

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set up the testing images and the training images. Each images pass through a resize and normalization function.

# Lidando com as Imagens

In [2]:
imagesTesting = []
labelsTesting = []
imagesTraining = []
labelsTraining = []
classLabels = ['barberry', 'bayberry', 'beach plum', 'bearberry', 'black berry', 'black cherry', 'blueberry', 'ceylon gooseberry', 'chokeberry', 'crowberry']

target_size = (224, 224)


def readTestingImages(i):
    os.chdir('/content/drive/MyDrive/Colab Notebooks/Images/Testing')

    for filename in os.listdir():
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read Image
            image = cv2.imread(filename)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Resize the image
            image_resized = cv2.resize(image_rgb, target_size)

            # Normalize the image
            image_normalized = image_resized / 255.0

            if i < 150:
              imagesTesting.append(image_normalized)

            # Extract label from filename
            labels = filename.rsplit("_", 1)
            label = labels[0].replace('_', ' ')

            # Get the index of the label in classLabels
            label_index = classLabels.index(label)

            if i < 150:
              labelsTesting.append(label_index)

            i += 1
            if i == 200:
              i = 0

i = 1
def readTrainingImages(i):
    os.chdir('/content/drive/MyDrive/Colab Notebooks/Images/Training')

    for filename in os.listdir():
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read Image
            image = cv2.imread(filename)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Resize the image
            image_resized = cv2.resize(image_rgb, target_size)

            # Normalize the image
            image_normalized = image_resized / 255.0

            if i < 50:
              imagesTraining.append(image_normalized)

            # Extract label from filename
            labels = filename.rsplit("_", 1)
            label = labels[0].replace('_', ' ')

            # Get the index of the label in classLabels
            label_index = classLabels.index(label)

            if i < 50:
              labelsTraining.append(label_index)

            i += 1
            if i == 200:
              i = 0

readTestingImages(1)
readTrainingImages(1)

Set up the CNN architecture

We start with a architecture of 7 layers. 2 are convolutional layers. 2 are max-pooling layers, a layer to flatten the previous output into a 1D vector, and 2 connected layers (dense layers).

**model = models.Sequential()**: this creates a sequential model, which is a linear stack of layers

**model.add(layers.Vonc2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))**: adds a 2D convolutional layer to the model with 32 filters. The number of filters determine the number of channels of the output feature maps. Each filter produces one feature map, and these feature maps collectively capture different aspects of the input data.

**model.add(layers.MaxPooling2D((2, 2)))**: adds a max-pooling layer to the model with a pool size of 2x2. Max-pooling reduces the spatial dimensions of the feature maps.

**model.add(layers.Flatten())**: this flattens the output of the previous layer into a 1D vector. This reshapes the multi-dimensional feature maps into a 1D vector that can be fed into the fully connected layer. Each element in the vector corresponds to a specific feature or activation in the feature map.

**model.add(layers.Dense(64, activation='relu'))**: This adds a fully connected (dense) layer with 64 units and the ReLU activation function.

**model.add(layers.Dense(10))**: This adds the final fully connected layer with 10 units. Since this is a classification problem with 10 classes, this layer will produce the output logits for each class.

# Arquitetura Resnet

In [3]:

def create_convulational_layer(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return MaxPool2D(3, strides=2, padding='same')(x)

def identity_block(tensor, f):
    x = create_convulational_layer(tensor, f)
    x = create_convulational_layer(x, f, 3)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)

    x = Add()([x, tensor])
    output = ReLU()(x)
    return output

def conv_block(tensor, f, s):
  x = create_convulational_layer(tensor, f)
  x = create_convulational_layer(x, f, 3, s)
  x = Conv2D(4*f, 1)(x)
  x = BatchNormalization()(x)

  shortcut = Conv2D(4*f, 1, strides=s)(tensor)
  shortcut = BatchNormalization()(shortcut)

  x = Add()([x, shortcut])
  output = ReLU()(x)
  return output

def create_resnet_block(x, f, r, s=2):
  x = conv_block(x, f, s)
  for _ in range(r-1):
    x = identity_block(x, f)
  return x

def build_resnet(input_shape, num_classes):
  input = Input(input_shape)

  # Primeira camada convolucional adicional
  x = create_convulational_layer(input, 64, 3, 2)

  # Segunda camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Terceira camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Quarta camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Quinta camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Sexta camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  x = create_resnet_block(x, 64, 3, 1)
  x = create_resnet_block(x, 128, 4)
  x = create_resnet_block(x, 256, 6)
  x = create_resnet_block(x, 512, 3)

  x = GlobalAvgPool2D()(x)

  x = Dropout(0.5)(x)

  output = Dense(num_classes, activation='softmax')(x)

  model = Model(input, output)
  return model

# Construindo Modelo

In [3]:
demo_resnet_model = Sequential()

pretrained_model_for_demo= tf.keras.applications.ResNet50(include_top=False, input_shape=(224,224,3), pooling='avg',classes=10, weights='imagenet')

for each_layer in pretrained_model_for_demo.layers:

        each_layer.trainable=False

demo_resnet_model.add(pretrained_model_for_demo)

demo_resnet_model.add(layers.Flatten())

demo_resnet_model.add(Dense(512, activation='relu'))

demo_resnet_model.add(Dense(10, activation='softmax'))

# Treinando Modelo

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
demo_resnet_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = demo_resnet_model.fit(np.array(imagesTraining), np.array(labelsTraining), epochs=30, validation_data=(np.array(imagesTesting), np.array(labelsTesting)))

Epoch 1/30
16/16 [==============================] - 15s 721ms/step - loss: 2.2376 - accuracy: 0.2380 - val_loss: 2.2572 - val_accuracy: 0.1860
Epoch 2/30
16/16 [==============================] - 7s 445ms/step - loss: 2.1668 - accuracy: 0.2000 - val_loss: 2.3129 - val_accuracy: 0.1640
Epoch 3/30
16/16 [==============================] - 7s 443ms/step - loss: 2.1492 - accuracy: 0.2120 - val_loss: 2.2527 - val_accuracy: 0.2127
Epoch 4/30
16/16 [==============================] - 7s 430ms/step - loss: 2.0901 - accuracy: 0.2500 - val_loss: 2.2732 - val_accuracy: 0.1980
Epoch 5/30
16/16 [==============================] - 7s 447ms/step - loss: 2.0956 - accuracy: 0.2500 - val_loss: 2.2671 - val_accuracy: 0.1607
Epoch 6/30
16/16 [==============================] - 7s 432ms/step - loss: 2.1101 - accuracy: 0.2660 - val_loss: 2.3003 - val_accuracy: 0.1420
Epoch 7/30
16/16 [==============================] - 7s 446ms/step - loss: 2.0976 - accuracy: 0.2500 - val_loss: 2.2746 - val_accuracy: 0.1647
Epoch

# Avaliação do Modelo

In [5]:
# Avaliação do modelo
test_loss, test_acc = model.evaluate(imagesTesting, labelsTesting)
print('Acurácia final do modelo: %.2f%%' % (test_acc * 100))

ValueError: ignored

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.05, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0.1, 30])
plt.legend(loc='lower right')

# Salvando Modelo

In [ ]:
# Salvar o modelo
model.save('/content/drive/MyDrive/CNN_SOURCE/Modelos/cnn_modelo_resnet.h5')